In [3]:
import pandas as pd

def txt_to_pd(input_txt): 
    mpi_processes = []
    real_time = []
    user_time = []
    sys_time = []

    # Open the text file
    with open(input_txt, 'r') as file:
        lines = file.readlines()

        # Initialize variables to hold current run and process numbers
        current_processes = None
        real = None
        user = None
        sys = None

        # Iterate through each line in the file
        for line in lines:
            if line.startswith(("MPI processes:", "OpenMP threads")):
                current_processes = int(line.split(":")[1].strip())
                mpi_processes.append(current_processes)
                
            elif line.startswith("real"):
                real = line.split()[-1].strip()
                minutes, seconds = real.split('m')
                seconds = seconds[:-1]
                # Convert minutes and seconds to seconds
                total_seconds = int(minutes) * 60 + float(seconds)
                real_time.append(total_seconds)
            
            elif line.startswith("user"):
                user = line.split()[-1].strip()
                minutes, seconds = user.split('m')
                seconds = seconds[:-1]
                # Convert minutes and seconds to seconds
                total_seconds = int(minutes) * 60 + float(seconds)
                user_time.append(total_seconds)
                
            
            elif line.startswith("sys"):
                sys = line.split()[-1].strip()
                minutes, seconds = sys.split('m')
                seconds = seconds[:-1]
                # Convert minutes and seconds to seconds
                total_seconds = int(minutes) * 60 + float(seconds)
                sys_time.append(total_seconds)

    data = {
    'MPI processes': mpi_processes,
    'Real time': real_time,
    'User time': user_time,
    'System time': sys_time}

    data = pd.DataFrame(data)

    return data


# MPI scaling: 


In [2]:
df = txt_to_pd('mpi_scaling.txt')
df

,MPI processes,Real time,User time,System time
0,2,1565.986,3117.178,1.431
1,4,733.536,2899.800,2.358
2,6,403.249,2384.972,3.700
3,8,477.725,3766.183,5.679
4,10,327.489,3216.891,7.314
...,...,...,...,...
123,248,37.150,3713.272,272.548
124,250,36.377,3617.018,279.145
125,252,37.071,3688.596,266.279
126,254,36.365,3613.696,271.832


### Plotting the execution time 

In [9]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# Plot the execution time 

fig = px.line(df, x='MPI processes', y='Real time', title="MPI Scaling in seconds")
total_processes = df['MPI processes'].max()
quarter_points = [int(total_processes / 4) * i for i in range(5)]  # [0, 1/4, 2/4, 3/4, 1]

# Set the x-axis tick marks and labels
fig.update_xaxes(
    tickvals=quarter_points,
    ticktext=[f'{int(p)}' for p in quarter_points]  # You can customize the format of the labels
)

# Annotate the quarter points with y-axis labels
for x in quarter_points:
    # Find the nearest MPI processes value to the quarter point
    nearest_idx = np.abs(df['MPI processes'] - x).idxmin()
    nearest_mpi_processes = df.loc[nearest_idx, 'MPI processes']
    nearest_real_time = df.loc[nearest_idx, 'Real time']
    fig.add_annotation(x=nearest_mpi_processes, y=nearest_real_time, text=str(nearest_real_time), showarrow=True, arrowhead=1)

# Show the plot
fig.show()

### Determining the speed up 

In [11]:
baseline : float = 1565.986
df['Speedup'] = baseline / df['Real time']
df 

,MPI processes,Real time,User time,System time,Speedup
0,2,1565.986,3117.178,1.431,1.000000
1,4,733.536,2899.800,2.358,2.134845
2,6,403.249,2384.972,3.700,3.883422
3,8,477.725,3766.183,5.679,3.278007
4,10,327.489,3216.891,7.314,4.781797
...,...,...,...,...,...
123,248,37.150,3713.272,272.548,42.153055
124,250,36.377,3617.018,279.145,43.048795
125,252,37.071,3688.596,266.279,42.242885
126,254,36.365,3613.696,271.832,43.063000


In [13]:
# Plot the execution time 

fig = px.line(df, x='MPI processes', y='Speedup', title="Speedup factor")
total_processes = df['MPI processes'].max()
quarter_points = [int(total_processes / 4) * i for i in range(5)]  # [0, 1/4, 2/4, 3/4, 1]

# Set the x-axis tick marks and labels
fig.update_xaxes(
    tickvals=quarter_points,
    ticktext=[f'{int(p)}' for p in quarter_points]  # You can customize the format of the labels
)

# Annotate the quarter points with y-axis labels
for x in quarter_points:
    # Find the nearest MPI processes value to the quarter point
    nearest_idx = np.abs(df['MPI processes'] - x).idxmin()
    nearest_mpi_processes = df.loc[nearest_idx, 'MPI processes']
    nearest_real_time = df.loc[nearest_idx, 'Speedup']
    fig.add_annotation(x=nearest_mpi_processes, y=nearest_real_time, text=str(nearest_real_time), showarrow=True, arrowhead=1)

# Show the plot
fig.show()

# OpenMP scaling

In [11]:
df = txt_to_pd('omp_scaling_socket.txt')
df = df.rename(columns={'MPI processes': 'OpenMP threads'})
df

,OpenMP threads,Real time,User time,System time
0,2,926.609,1834.458,3.466
1,4,856.403,3384.166,3.898
2,6,462.500,2735.666,2.575
3,8,553.890,4359.398,3.365
4,10,367.810,3611.656,2.393
5,12,459.878,5404.400,3.150
6,14,339.712,4645.499,2.659
7,16,476.156,7436.214,3.601
8,18,310.705,5445.415,2.602
9,20,302.655,5891.768,2.612


### Plotting the execution time

In [13]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# Plot the execution time 

fig = px.line(df, x='OpenMP threads', y='Real time', title="OpenMP Scaling in seconds")
total_processes = df['OpenMP threads'].max()
quarter_points = [int(total_processes / 4) * i for i in range(5)]  # [0, 1/4, 2/4, 3/4, 1]

# Set the x-axis tick marks and labels
fig.update_xaxes(
    tickvals=quarter_points,
    ticktext=[f'{int(p)}' for p in quarter_points]  # You can customize the format of the labels
)

# Annotate the quarter points with y-axis labels
for x in quarter_points:
    # Find the nearest MPI processes value to the quarter point
    nearest_idx = np.abs(df['OpenMP threads'] - x).idxmin()
    nearest_mpi_processes = df.loc[nearest_idx, 'OpenMP threads']
    nearest_real_time = df.loc[nearest_idx, 'Real time']
    fig.add_annotation(x=nearest_mpi_processes, y=nearest_real_time, text=str(nearest_real_time), showarrow=True, arrowhead=1)

# Show the plot
fig.show()

### Determining the speedup 

In [15]:
baseline : float = 926.609
df['Speedup'] = baseline / df['Real time']
df 

,OpenMP threads,Real time,User time,System time,Speedup
0,2,926.609,1834.458,3.466,1.000000
1,4,856.403,3384.166,3.898,1.081978
2,6,462.500,2735.666,2.575,2.003479
3,8,553.890,4359.398,3.365,1.672912
4,10,367.810,3611.656,2.393,2.519260
5,12,459.878,5404.400,3.150,2.014902
6,14,339.712,4645.499,2.659,2.727631
7,16,476.156,7436.214,3.601,1.946020
8,18,310.705,5445.415,2.602,2.982279
9,20,302.655,5891.768,2.612,3.061601


In [16]:
# Plot the execution time 

fig = px.line(df, x='OpenMP threads', y='Speedup', title="Speedup factor")
total_processes = df['OpenMP threads'].max()
quarter_points = [int(total_processes / 4) * i for i in range(5)]  # [0, 1/4, 2/4, 3/4, 1]

# Set the x-axis tick marks and labels
fig.update_xaxes(
    tickvals=quarter_points,
    ticktext=[f'{int(p)}' for p in quarter_points]  # You can customize the format of the labels
)

# Annotate the quarter points with y-axis labels
for x in quarter_points:
    # Find the nearest MPI processes value to the quarter point
    nearest_idx = np.abs(df['OpenMP threads'] - x).idxmin()
    nearest_mpi_processes = df.loc[nearest_idx, 'OpenMP threads']
    nearest_real_time = df.loc[nearest_idx, 'Speedup']
    fig.add_annotation(x=nearest_mpi_processes, y=nearest_real_time, text=str(nearest_real_time), showarrow=True, arrowhead=1)

# Show the plot
fig.show()